In [ ]:
#| default_exp utils

# fcst_campaign utils module

> Some generic tools

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
IN_COLAB = 'google.colab' in str(get_ipython())
if IN_COLAB:
  !pip install git+https://github.com/pete88b/nbdev_colab_helper.git
  from nbdev_colab_helper.core import *
  project_name = 'forecast_action'
  init_notebook(project_name)

In [ ]:
#| hide
from nbdev.showdoc import *

if IN_COLAB:
  !pip install timezonefinder



In [ ]:
#| export
import pytz
from datetime import datetime
from timezonefinder import TimezoneFinder

In [ ]:
#| export
def get_state_capitals(geo_entry):
    # Maps of country to a list of its state capitals
    capitals = {
        "USA": ["Washington, D.C.", "Austin, TX", "Sacramento, CA", "New York, NY", "Chicago, IL", "Denver, CO", "Kanas City, MO"],
        "India": ["New Delhi", "Mumbai", "Kolkata", "Chennai", "Hyderabad", "Pune"]
    }

    # Check if the geo_entry is a recognized country and return its capitals
    if geo_entry in capitals:
        return capitals[geo_entry]
    else:
        # Return an empty list or a meaningful message if no data is available
        return []



In [ ]:
#| export

import geopy.geocoders
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
import certifi
import ssl
import time

def get_lat_lon(geo_input):
    """
    Convert a geographical input (city, state, or country) into a list of latitude-longitude pairs.
    """
    ctx = ssl.create_default_context(cafile=certifi.where())
    geopy.geocoders.options.default_ssl_context = ctx
    
    geolocator = Nominatim(user_agent="fcst_act/vijay.manghnani@gmail.com",timeout=10)
    # Use RateLimiter to avoid hitting usage limits
    geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1)

    lat_lon_list = []

    # Example of handling different types of geographical inputs
    if isinstance(geo_input, str):  # Single city, state, or country
        location = geolocator.geocode(geo_input)
        if location:
            lat_lon_list.append((location.latitude, location.longitude))
    elif isinstance(geo_input, list):  # List of cities
        for item in geo_input:
            location = geolocator.geocode(item)
            if location:
                lat_lon_list.append((location.latitude, location.longitude))
            time.sleep(1)

    return lat_lon_list

In [ ]:
#| export
def create_geo_query_list(company_df, company_name):
    """
    For a given company, generate a list of latitude-longitude pairs based on its 'Geo' field.
    """
    # Find the 'Geo' entry for the company
    geo_entry = company_df[company_df['Company'] == company_name]['Geo'].iloc[0]
    #print(geo_entry)

    country_names = ['USA', 'India']
    ## Get a list of cities
    if geo_entry in country_names:
      city_names = get_state_capitals(geo_entry)
    else:
      city_names = [geo_entry]

    # Handling different Geo formats (e.g., city, state, country)
    # This is a placeholder; implementation depends on your data and needs
    lat_lon_list = get_lat_lon(city_names)

    return lat_lon_list, city_names

In [ ]:
#| export
def round_to_grid(lat, lon, resolution=0.25):
    """Round latitude and longitude to the nearest grid resolution."""
    rounded_lat = round(lat / resolution) * resolution
    rounded_lon = round(lon / resolution) * resolution
    return rounded_lat, rounded_lon
'''
# Example: Rounding the coordinates for Chicago
chicago_lat, chicago_lon = 41.8781, -87.6298
rounded_lat, rounded_lon = round_to_grid(chicago_lat, chicago_lon)

print(f"Rounded Coordinates for Chicago: Latitude {rounded_lat}, Longitude {rounded_lon}")
'''

'\n# Example: Rounding the coordinates for Chicago\nchicago_lat, chicago_lon = 41.8781, -87.6298\nrounded_lat, rounded_lon = round_to_grid(chicago_lat, chicago_lon)\n\nprint(f"Rounded Coordinates for Chicago: Latitude {rounded_lat}, Longitude {rounded_lon}")\n'

In [ ]:
#| export
def get_local_peak_hours(lat_lons):
    tf = TimezoneFinder()
    peak_hours_info = []

    for lat, lon in lat_lons:
        # Find the time zone for the given lat-lon
        tz_str = tf.timezone_at(lat=lat, lng=lon)
        tz = pytz.timezone(tz_str)

        # Define peak hours in UTC
        utc_peak_start = datetime.utcnow().replace(hour=10, minute=0, second=0, microsecond=0)
        utc_peak_end = datetime.utcnow().replace(hour=17, minute=0, second=0, microsecond=0)

        # Convert peak hours to local time
        local_peak_start = utc_peak_start.astimezone(tz)
        local_peak_end = utc_peak_end.astimezone(tz)

        peak_hours_info.append({
            "lat_lon": (lat, lon),
            "time_zone": tz_str,
            "local_peak_start": local_peak_start.strftime("%Y-%m-%d %H:%M:%S"),
            "local_peak_end": local_peak_end.strftime("%Y-%m-%d %H:%M:%S"),
        })

    return peak_hours_info

# Example usage
lat_lons = [(40.7128, -74.0060), (28.6139, 77.2090)]  # New York, New Delhi
peak_hours = get_local_peak_hours(lat_lons)
for info in peak_hours:
    print(info)


{'lat_lon': (40.7128, -74.006), 'time_zone': 'America/New_York', 'local_peak_start': '2024-03-11 10:00:00', 'local_peak_end': '2024-03-11 17:00:00'}
{'lat_lon': (28.6139, 77.209), 'time_zone': 'Asia/Kolkata', 'local_peak_start': '2024-03-11 19:30:00', 'local_peak_end': '2024-03-12 02:30:00'}


C:\Users\vjltd\AppData\Local\Temp\ipykernel_23420\3142825042.py:12: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  utc_peak_start = datetime.utcnow().replace(hour=10, minute=0, second=0, microsecond=0)
C:\Users\vjltd\AppData\Local\Temp\ipykernel_23420\3142825042.py:13: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  utc_peak_end = datetime.utcnow().replace(hour=17, minute=0, second=0, microsecond=0)


In [ ]:
#| export
def get_timezone_offset(lat, lon, date=datetime.now()):
    """
    Returns the timezone offset in hours for a given latitude, longitude, and date.

    :param lat: Latitude of the location
    :param lon: Longitude of the location
    :param date: Date for which to find the timezone offset, default is current date
    :return: Timezone offset in hours from UTC
    """
    tf = TimezoneFinder()
    timezone_str = tf.timezone_at(lat=lat, lng=lon)  # Find the timezone name
    if timezone_str is None:
        return None  # Could not determine the timezone
    timezone = pytz.timezone(timezone_str)

    # Calculate offset in hours, considering daylight saving time
    offset = timezone.utcoffset(date)
    offset_hours = offset.total_seconds() / 3600
    return offset_hours

# Example usage
lat, lon = 40.7128, -74.0060  # New York City
offset_hours = get_timezone_offset(lat, lon)
print(f"Timezone offset for latitude {lat}, longitude {lon} is {offset_hours} hours.")

Timezone offset for latitude 40.7128, longitude -74.006 is -4.0 hours.


In [ ]:
#| export
def calculate_search_radius(city_lat):
    # Constants
    km_per_degree_latitude = 111  # Approximate km covered by one degree of latitude
    longitude_difference = 0.25  # Grid cell size in degrees (longitude)

    # Calculating the East-West length of the grid cell at the given latitude
    E_W_length_km = km_per_degree_latitude * math.cos(math.radians(city_lat)) * longitude_difference

    # Calculating the radius as half the E-W length
    radius_km = E_W_length_km / 2
    return radius_km

In [ ]:
#| hide
if IN_COLAB:
    from nbdev.export import notebook2script
    notebook2script()
else:
    import nbdev; nbdev.nbdev_export()